In [ ]:
 #@title CHAT WITH ME
print("********* WELCOME TO CHAT WITH ME *********")

********* WELCOME TO CHAT WITH ME *********


In [ ]:
#@title MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title VIDEO TO TEXT
import os
from os.path import exists, join, basename, splitext

if not exists('deepspeech-0.6.1-models'):
  !apt-get install -qq sox
  !pip install -q deepspeech-gpu==0.6.1 youtube-dl
  !wget https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
  !tar xvfz deepspeech-0.6.1-models.tar.gz
  
from IPython.display import YouTubeVideo
#YOUTUBE_ID = '2AFpAATHXtc'  
YOUTUBE_ID = '4FDud9Lj5HY'
YouTubeVideo(YOUTUBE_ID)
!rm -rf *.wav
!youtube-dl --extract-audio --audio-format wav --output "test.%(ext)s" https://www.youtube.com/watch\?v\={YOUTUBE_ID}
!deepspeech --model deepspeech-0.6.1-models/output_graph.pbmm --lm deepspeech-0.6.1-models/lm.binary --trie deepspeech-0.6.1-models/trie --audio test.wav

[youtube] 4FDud9Lj5HY: Downloading webpage
[download] Destination: test.webm
[download] 100% of 1.18MiB in 00:21
[ffmpeg] Destination: test.wav
Deleting original file test.webm (pass -k to keep)
Loading model from file deepspeech-0.6.1-models/output_graph.pbmm
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.1-0-g3df20fe
2022-06-21 04:41:57.718460: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-21 04:41:57.719635: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2022-06-21 04:41:57.727602: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-21 04:41:57.727658: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2e3c122107b8): /proc/driver/nvidia/version does not exi

In [ ]:
#@title IMAGE TO TEXT
import cv2
# !pip install pytesseract
# !pip install Pillow 
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract


from PIL import Image
import pytesseract 
from IPython.display import display # to display images
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')
#from google.colab import files
#uploaded = files.upload()

image=Image.open('/content/drive/MyDrive/123.png','r')
display(image)
text = pytesseract.image_to_string(image)
print(text)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title SENTIMENT ANALYZER
import pandas as pd
import seaborn as sns
!pip install -U nltk
sns.set(rc={'figure.figsize':(30,1)})

def visualise_sentiments(data):
  sns.heatmap(pd.DataFrame(data).set_index("Sentence").T,center=0, annot=True, cmap = "PiYG")
sentence =input()
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(sentence)
visualise_sentiments({
    "Sentence":["SENTENCE"] + sentence.split(),
    "Sentiment":[sid.polarity_scores(sentence)["compound"]] + [sid.polarity_scores(word)["compound"] for word in sentence.split()]
})
from textblob import TextBlob
TextBlob(sentence).sentiment



In [ ]:
#@title CHAT WITH ME
import nltk
import numpy as np
import random
import string # to process standard python strings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


f=open('/content/drive/My Drive/ai.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('omw-1.4')
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey","hai")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
        
   
GREETING_EXCUSES = ["Come again please", "I can't understand you", "Please say one more time", "Pardon", "I don't know what you are saying"]
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+random.choice(GREETING_EXCUSES)
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response


GREETING_ENDSTATEMENTS= ("Chat With Me: Bye! take care..", "Chat With Me: Bye! I hope to see you soon", "Chat With Me: Bye! stay home stay safe","Be safe, See you soon")
GREETING_THANKRESPONSE= ("Chat With Me: You are welcome", "Chat With Me: No mention", "Chat With Me: Thanks is banned here", "Chat With Me: Don't thank a friend buddy","Chat With Me: Keep it yourself")

flag=True
print("Chat With Me: My name is Chat With Me. I will answer your queries about Artificial intelligence. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            thankresponse=random.choice(GREETING_THANKRESPONSE)
            print(thankresponse)
        else:
            if(greeting(user_response)!=None):
                print("Chat With Me: "+greeting(user_response))
            else:
                print("Chat With Me: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        endstatements=random.choice(GREETING_ENDSTATEMENTS)
        print(endstatements)    